In [37]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

In [18]:
dataset = pd.read_csv('./dataset.csv')

In [19]:
dataset.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            381 non-null    object 
 1   Gender             376 non-null    object 
 2   Married            381 non-null    object 
 3   Dependents         373 non-null    object 
 4   Education          381 non-null    object 
 5   Self_Employed      360 non-null    object 
 6   ApplicantIncome    381 non-null    int64  
 7   CoapplicantIncome  381 non-null    float64
 8   LoanAmount         381 non-null    float64
 9   Loan_Amount_Term   370 non-null    float64
 10  Credit_History     351 non-null    float64
 11  Property_Area      381 non-null    object 
 12  Loan_Status        381 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 38.8+ KB


In [21]:
dataset.drop('Loan_ID', axis=1, inplace=True)

In [22]:
dataset.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,381.000000,381.000000,381.000000,370.000000,351.000000
mean,3579.845144,1277.275381,104.986877,340.864865,0.837607
std,1419.813818,2340.818114,28.358464,68.549257,0.369338
min,150.000000,0.000000,9.000000,12.000000,0.000000
25%,2600.000000,0.000000,90.000000,360.000000,1.000000
50%,3333.000000,983.000000,110.000000,360.000000,1.000000
75%,4288.000000,2016.000000,127.000000,360.000000,1.000000
max,9703.000000,33837.000000,150.000000,480.000000,1.000000


In [23]:
dataset.isna().sum()

Gender                5
Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     11
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64

In [24]:
dataset.dropna(axis=0, inplace=True)

In [25]:
categorical_columns = []
numerical_columns = []
for i in dataset.columns:
    if(dataset[i].dtype == 'object'):
        categorical_columns.append(i)
    else:
        numerical_columns.append(i)

In [26]:
for i in categorical_columns:
    print(f'{i} : {dataset[i].unique()}')

Gender : ['Male' 'Female']
Married : ['Yes' 'No']
Dependents : ['1' '0' '2' '3+']
Education : ['Graduate' 'Not Graduate']
Self_Employed : ['No' 'Yes']
Property_Area : ['Rural' 'Urban' 'Semiurban']
Loan_Status : ['N' 'Y']


In [27]:
dataset['Dependents'].replace({'0':0, '1':1, '2':2, '3+':3}, inplace=True)

In [28]:
encoder_map = {}
for i in categorical_columns:
    if(i != 'Dependents'):
        le = LabelEncoder()
        dataset[i] = le.fit_transform(dataset[i])
        encoder_map[i] = le

In [29]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, )

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [34]:
models = {
    'LogisticRegression' : LogisticRegression(max_iter=1000),
    'KNeighborsClassifier' : KNeighborsClassifier(),
    'SVC' : SVC(),
    'GaussianNB' : GaussianNB(),
    'DecisionTreeClassifier' : DecisionTreeClassifier(),
    'RandomForestClassifier' : RandomForestClassifier()
}
param_grids = {
    'RandomForestClassifier': {
        'n_estimators': [100, 300, 500, 800, 1000],
        'max_depth': [10, 20, 30, None]
    },
    'SVC': {
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto']
    },
    'LogisticRegression': {
        'C': [0.001, 0.01, 0.1, 1, 10, 100]
    },
    'KNeighborsClassifier': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    },
    'DecisionTreeClassifier': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'GaussianNB': {}
}

for key, value in models.items():
    grid_search = GridSearchCV(estimator=value, param_grid=param_grids[key],cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X=X_train, y=y_train)
    print(f'{key}({grid_search.best_params_}): {grid_search.best_score_}')

/Users/ayushrajsingh/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression({'C': 100}): 0.8353260869565217
KNeighborsClassifier({'algorithm': 'auto', 'n_neighbors': 9, 'weights': 'distance'}): 0.6885869565217391
SVC({'C': 1, 'gamma': 'auto'}): 0.7056159420289855
GaussianNB({}): 0.8309782608695653
DecisionTreeClassifier({'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 5}): 0.7574275362318841
RandomForestClassifier({'max_depth': 10, 'n_estimators': 1000}): 0.8309782608695653


In [51]:
model = RandomForestClassifier(max_depth=10, n_estimators=1000)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=1000)

In [52]:
y_pred = model.predict(X_test)

In [53]:
acc = accuracy_score(y_pred=y_pred, y_true=y_test)
cm = confusion_matrix(y_pred=y_pred, y_true=y_test)
print(f'Accuracy : {acc}')
print(f'Confusion Matrix :\n{cm}')

Accuracy : 0.8441558441558441
Confusion Matrix :
[[ 9 10]
 [ 2 56]]
